<a href="https://colab.research.google.com/github/Ahmed-Shatla/NLP-data-cleaning/blob/main/Cleaning_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk

#downloading

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

#Reading data

In [3]:
from nltk.corpus import twitter_samples
pos_tweets = twitter_samples.strings('positive_tweets.json')
neg_tweets = twitter_samples.strings('negative_tweets.json')

#data cleaning

In [4]:
import re

In [5]:
def cleanText(text):
  text= re.sub(r'#[\S|_]*','',text) #hashtage removing
  text= re.sub(r'@[\S|_]*','',text) #username removing
  text= re.sub(r'https?:\/\/\S+','',text) #hyperlink removing
  text = re.sub(r'\W',' ',text) #remove any emotions
  text = re.sub(r'\d+','',text) #remove any standalone digits
  text = re.sub(r'^\s+','',text)#remove spaces that at start of the sentences
  text = re.sub(r'\s+$','',text)#remove spaces that at end of the sentences

  return text

In [6]:
cleanText(pos_tweets[0])

'for being top engaged members in my community this week'

In [7]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from nltk.stem import PorterStemmer
def process_on_tweets(tweets):
  result=[]
  for tweet in tweets:
    tweet = cleanText(tweet)
    tweet = tweet.split()
    tweet=[word for word in tweet if word.lower() not in stop_words]

    ps = PorterStemmer()
    tweet=[ps.stem(word) for word in tweet]

    result.append(tweet)
  return result

In [9]:
positive_tweets = process_on_tweets(pos_tweets)
negative_tweets = process_on_tweets(neg_tweets)

#Frequinces
### (word,label):freq

In [10]:
def build_freq(tweets,label):
  freq ={}
  for tweet in tweets:
    for word in tweet:
      key = (word,label)
      if key in freq:
        freq[key]+=1
      else:
        freq[key]=1

  return freq

In [11]:
pos_freq = build_freq(positive_tweets,1)
neg_freq = build_freq(negative_tweets,0)

#building features

In [30]:
def build_features(tweets,p_freq,n_freq):
  features=[]
  temp_vec = []

  for tweet in tweets:
    temp_vec=[0]*3   #[0,0,0]
    p_count = 0
    n_count = 0
    temp_vec[0]=1   #bias = 1
    for word in tweet:
      p_count+=p_freq.get((word,1),1)
      n_count+=n_freq.get((word,0),0)

    temp_vec[1]=p_count
    temp_vec[2]=n_count


    features.append(temp_vec)
  return features

In [31]:
pos_features = build_features(positive_tweets,pos_freq,neg_freq)
neg_features = build_features(negative_tweets,pos_freq,neg_freq)

In [32]:
pos_features[0]

[1, 173, 73]

In [33]:
len(pos_features)

5000

In [34]:
import numpy as np
pos_y = np.ones((len(pos_features),1))
neg_y = np.zeros((len(neg_features),1))

In [35]:
X=[]
y = []
for i in range(len(pos_features)):
  X.append(pos_features[i])
  X.append(neg_features[i])
  y.append(pos_y[i])
  y.append(neg_y[i])
